**Importing libraries**

In [1]:
import pandas as pd
import numpy as np
import cufflinks as cf # to link series to iplot for eda plotting


from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import string
import re
#string.punctuation

**Fake news dataframe**

In [2]:
#fake news (not including claims and tweets)
fake_df = pd.read_csv('https://raw.githubusercontent.com/cuilimeng/CoAID/master/07-01-2020/NewsFakeCOVID-19.csv')
fake_df.head(2)


,Unnamed: 0,type,fact_check_url,archieve,news_url,news_url2,news_url3,news_url4,news_url5,title,newstitle,content,abstract,publish_date,meta_keywords
0,623,post,https://www.factcheck.org/2020/07/face-mask-ex...,http://archive.is/LC508,https://www.facebook.com/tednugent/photos/a.10...,NaN,NaN,NaN,NaN,A “face mask exempt card” shared on social med...,"""Ted Nugent""",snag limbrats that daddy 22s out of the toweri...,ted nugent added a new photo ..,NaN,""""""
1,624,post,https://www.factcheck.org/2020/07/video-presen...,http://archive.is/l3qLv,https://www.facebook.com/jeff.neff.528/videos/...,NaN,NaN,NaN,NaN,A test on video with a gas detector shows that...,"""Jeff""",see more of jeff neff on facebook.,face masks you decide. this is worth a few min...,NaN,""""""


In [3]:
fake_df.columns

Index(['Unnamed: 0', 'type', 'fact_check_url', 'archieve', 'news_url',
       'news_url2', 'news_url3', 'news_url4', 'news_url5', 'title',
       'newstitle', 'content', 'abstract', 'publish_date', 'meta_keywords'],
      dtype='object')

In [4]:
fake_df.shape

(266, 15)

In [5]:
# Changing datatype for content column
fake_df['content'] = fake_df['content'].values.astype('U')

In [6]:
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [7]:
# Defining function to get top n words
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

common_words2 = get_top_n_words(fake_df['content'], 20)

for word, freq in common_words2:
    print(word, freq)

    df2 = pd.DataFrame(common_words2, columns = ['content' , 'count'])
df2.groupby('content').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in review after removing stop words from fake dataset')

facebook 124
nan 42
want 26
covid 24
like 23
19 22
coronavirus 22
join 22
ve 21
javascript 19
browser 18
detected 17
disabled 17
proceed 17
legacy 17
twitter 17
president 17
trump 17
people 15
health 14


**Real news dataframe**

In [8]:
# real news (not including claims and tweets)
real_df = pd.read_csv('https://raw.githubusercontent.com/cuilimeng/CoAID/master/07-01-2020/NewsRealCOVID-19.csv')
real_df

,Unnamed: 0,type,fact_check_url,news_url,title,newstitle,content,abstract,publish_date,meta_keywords
0,1555,article,healthline.com,https://www.healthline.com/health-news/how-to-...,How to Enjoy the Fourth of July Weekend Amid t...,"""How to Enjoy Fourth of July Weekend Amid the ...",experts recommend everyone avoid large gatheri...,experts recommend everyone avoid large gatheri...,NaN,"""COVID19 coronavirus prevention advice July 4 ..."
1,1556,article,healthline.com,https://www.healthline.com/health-news/cdc-exp...,CDC Expands List of Those with Higher COVID-19...,"""CDC Expands List of Those with Higher COVID-1...",experts say the key is to stay in the know and...,officials at the centers for disease control a...,NaN,"""CDC list expanded COVID-19 coronavirus higher..."
2,1557,article,healthline.com,https://www.healthline.com/health-news/feds-wa...,Feds Warn That Face Mask ‘Exemption’ Cards Are...,"""Feds Warn That Face Mask ‘Exemption’ Cards Ar...",some people have reportedly been presenting fa...,reports of people presenting cards that claim ...,NaN,"""covid 19, coronavirus, face mask exemption ca..."
3,1558,article,healthline.com,https://www.healthline.com/health-news/experts...,Experts Confident Biden’s COVID-19 Response Co...,"""Experts Confident Biden's COVID-19 Response C...",medical experts say they believe bidens plan t...,medical experts say they believe bidens plan t...,NaN,"""covid 19, coronavirus, testing and tracing"""
4,1559,article,healthline.com,https://www.healthline.com/health-news/heres-e...,Here’s Exactly Where We Are with Vaccines and ...,"""Where We're at with Vaccines and Treatments f...",scientists around the world are working on a n...,scientists around the world are working on a n...,NaN,"""covid19 coronavirus outbreak research progres..."
...,...,...,...,...,...,...,...,...,...,...
1122,10054,article,https://www.politifact.com/factchecks/2020/may...,https://www.indianapolismonthly.com/lifestyle/...,Says researchers found a “fabric that kills co...,"""403 Forbidden""",NaN,NaN,NaN,""""""
1123,10055,post,https://www.politifact.com/factchecks/2020/jun...,https://www.youtube.com/watch?v=Fpr4D5Bicnw&fe...,“With retail people are moving around and you ...,"""Coronavirus Briefing: NC Gov. Roy Cooper (05/...",NaN,north carolina governor roy cooper along with ...,5/12/20,"""Coronavirus, COVID-19, NC, North Carolina, Go..."
1124,10056,article,https://www.politifact.com/factchecks/2020/jun...,https://www.azcentral.com/story/news/politics/...,Says President Donald Trump “took more than a ...,"""Joe Biden pledges increased funding for India...",with the navajo nation leading the nation in t...,joe biden said he would uphold tribal sovereig...,NaN,""""""
1125,10057,post,https://www.politifact.com/factchecks/2020/jun...,https://twitter.com/realDonaldTrump/status/127...,As of June 23 the COVID-19 death rate is “way ...,"""""",we 've detected that javascript is disabled in...,NaN,NaN,""""""


In [9]:
real_df.columns

Index(['Unnamed: 0', 'type', 'fact_check_url', 'news_url', 'title',
       'newstitle', 'content', 'abstract', 'publish_date', 'meta_keywords'],
      dtype='object')

In [10]:
real_df.shape

(1127, 10)

In [11]:
#df.fillna('0')
real_df['content'] = real_df['content'].values.astype('U')

In [12]:
fake_df = fake_df.dropna(axis=0, subset=['news_url'])

In [13]:
print(fake_df.news_url)

0      https://www.facebook.com/tednugent/photos/a.10...
1      https://www.facebook.com/jeff.neff.528/videos/...
2      https://twitter.com/SexCounseling/status/12764...
3      https://www.wakingtimes.com/2020/05/15/bill-ga...
4      https://www.facebook.com/photo.php?fbid=424048...
                             ...                        
261    https://www.facebook.com/BrokerCam/posts/11001...
262    https://www.facebook.com/watch/?v=619001568957940
263    https://www.facebook.com/photo.php?fbid=273325...
264    https://www.facebook.com/CaptRandall/posts/102...
265    https://themissouritimes.com/opinion-pandemics...
Name: news_url, Length: 252, dtype: object


In [14]:
# unigram words after removing stopwords

common_words3 = get_top_n_words(real_df['content'], 20)
for word, freq in common_words3:
    print(word, freq)

    df3 = pd.DataFrame(common_words3, columns = ['content' , 'count'])
df3.groupby('content').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in review after removing stop words from real dataset')

covid 876
19 871
protect 397
using 388
service 370
online 367
website 356
security 355
attacks 354
people 350
data 284
health 262
coronavirus 244
information 238
new 191
disease 181
virus 171
pandemic 168
available 166
time 164


**fake_df has 5 more columns than real_df.** <br>
**namely:**<Br>
**news_url2, news_url3, news_url4, news_url5, 'archieve'**

**Checking on what is in the url 1-5 columns because fake news has more columns than real news**

In [15]:
fake_df.iloc[8]

Unnamed: 0                                                      631
type                                                           post
fact_check_url    https://www.factcheck.org/2020/06/fake-aoc-twe...
archieve                                    http://archive.is/Z8dNh
news_url          https://twitter.com/PhilMcCrackin44/status/127...
news_url2         https://twitter.com/JmeDubya/status/1275512088...
news_url3         https://www.instagram.com/p/CByRGdZAd1w/?utm_s...
news_url4         https://www.instagram.com/p/CByAnHJAPiU/?utm_s...
news_url5         https://www.facebook.com/2066881336880155/post...
title             Claim that Rep. Alexandria Ocasio-Cortez tweet...
newstitle                                                        ""
content           we 've detected that javascript is disabled in...
abstract                                                        NaN
publish_date                                                    NaN
meta_keywords                                   

In [16]:
#fake_df.head(9)
fake_df.iloc[8,11]

"we 've detected that javascript is disabled in your browser. would you like to proceed to legacy twitter. "

**Dropping extra columns in fake_df**

In [17]:
fake_df = fake_df.drop(['news_url2', 'news_url3', 'news_url4', 'news_url5', 'archieve'] , axis='columns')

In [18]:
fake_df.head(2)

,Unnamed: 0,type,fact_check_url,news_url,title,newstitle,content,abstract,publish_date,meta_keywords
0,623,post,https://www.factcheck.org/2020/07/face-mask-ex...,https://www.facebook.com/tednugent/photos/a.10...,A “face mask exempt card” shared on social med...,"""Ted Nugent""",snag limbrats that daddy 22s out of the toweri...,ted nugent added a new photo ..,NaN,""""""
1,624,post,https://www.factcheck.org/2020/07/video-presen...,https://www.facebook.com/jeff.neff.528/videos/...,A test on video with a gas detector shows that...,"""Jeff""",see more of jeff neff on facebook.,face masks you decide. this is worth a few min...,NaN,""""""


**Checking for nonsensical entries containing = 54** <br>
**Percentage of total datasets that is nonsense: 20% (there are more but i didnt count every one)**

In [19]:
fake_df.content.value_counts()

nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         27
do you want to join facebook ?.                                                                                                                                                                                                                                                                                                                                                                                                                                                                             17
we 've det

In [20]:
# Use below code to search the column for keywords like javascript/facebook/twitter to sieve out
# the nonsensical rows
#fake_df[fake_df['content'].str.contains("facebook")]

**Adding labels to both real and fake datasets: 1 means fake news, 0 means real news**

In [21]:
fake_df['label'] = 1
real_df['label'] = 0
real_df.head()

,Unnamed: 0,type,fact_check_url,news_url,title,newstitle,content,abstract,publish_date,meta_keywords,label
0,1555,article,healthline.com,https://www.healthline.com/health-news/how-to-...,How to Enjoy the Fourth of July Weekend Amid t...,"""How to Enjoy Fourth of July Weekend Amid the ...",experts recommend everyone avoid large gatheri...,experts recommend everyone avoid large gatheri...,NaN,"""COVID19 coronavirus prevention advice July 4 ...",0
1,1556,article,healthline.com,https://www.healthline.com/health-news/cdc-exp...,CDC Expands List of Those with Higher COVID-19...,"""CDC Expands List of Those with Higher COVID-1...",experts say the key is to stay in the know and...,officials at the centers for disease control a...,NaN,"""CDC list expanded COVID-19 coronavirus higher...",0
2,1557,article,healthline.com,https://www.healthline.com/health-news/feds-wa...,Feds Warn That Face Mask ‘Exemption’ Cards Are...,"""Feds Warn That Face Mask ‘Exemption’ Cards Ar...",some people have reportedly been presenting fa...,reports of people presenting cards that claim ...,NaN,"""covid 19, coronavirus, face mask exemption ca...",0
3,1558,article,healthline.com,https://www.healthline.com/health-news/experts...,Experts Confident Biden’s COVID-19 Response Co...,"""Experts Confident Biden's COVID-19 Response C...",medical experts say they believe bidens plan t...,medical experts say they believe bidens plan t...,NaN,"""covid 19, coronavirus, testing and tracing""",0
4,1559,article,healthline.com,https://www.healthline.com/health-news/heres-e...,Here’s Exactly Where We Are with Vaccines and ...,"""Where We're at with Vaccines and Treatments f...",scientists around the world are working on a n...,scientists around the world are working on a n...,NaN,"""covid19 coronavirus outbreak research progres...",0


**checking the top real news sources**

In [22]:
real_df.news_url.value_counts()

https://www.sciencedaily.com/releases/2020/05/200514115834.htm                                                    2
https://www.sciencedaily.com/releases/2020/06/200611152453.htm                                                    2
https://www.sciencedaily.com/releases/2020/06/200610135015.htm                                                    2
https://www.sciencedaily.com/releases/2020/05/200518144915.htm                                                    2
https://www.sciencedaily.com/releases/2020/05/200522095506.htm                                                    2
                                                                                                                 ..
https://www.cdc.gov/coronavirus/2019-ncov/php/contact-tracing/jurisdiction-specific-contact-tracing-tools.html    1
https://www.webmd.com/lung/news/20200615/new-covid-19-outbreak-locks-down-beijing-district                        1
https://www.webmd.com/lung/news/20200506/expert-warns-u-s-might-not-lowe

In [23]:
real_df = real_df.dropna(axis=0, subset=['news_url'])

In [24]:
# making pandas display as many rows as I want
pd.options.display.max_columns = None
pd.options.display.max_rows = None
#df.head(100)

In [25]:
print(real_df.news_url)

0       https://www.healthline.com/health-news/how-to-...
1       https://www.healthline.com/health-news/cdc-exp...
2       https://www.healthline.com/health-news/feds-wa...
3       https://www.healthline.com/health-news/experts...
4       https://www.healthline.com/health-news/heres-e...
5       https://www.healthline.com/health-news/best-ma...
6       https://www.healthline.com/health-news/covid19...
7       https://www.healthline.com/health-news/what-to...
8       https://www.healthline.com/health-news/fightin...
9       https://www.healthline.com/health-news/travel-...
10      https://www.healthline.com/health-news/learn-t...
11      https://www.healthline.com/health-news/people-...
12      https://www.healthline.com/health-news/flu-all...
13      https://www.healthline.com/health-news/fda-say...
14      https://www.healthline.com/health-news/why-are...
15      https://www.healthline.com/health-news/more-yo...
16      https://www.healthline.com/health-news/no-face...
17      https:

In [26]:
# real_df['news_url_root'] = ""
# print(real_df.shape[0])
# #print(real_df.news_url[1112])
# #print(real_df['news_url_root'])
# for i,j in range(real_df.iterrows()):
#     real_df['news_url_root'][i] = real_df['news_url'][i].split(".com", 1)
# # real_df.news_url_root = real_df.news_url.split(".com", 1)
# real_df.head()

**combining real and fake news dataframe**

In [27]:
df = pd.concat([fake_df,real_df], axis=0, ignore_index=True)

In [28]:
df

,Unnamed: 0,type,fact_check_url,news_url,title,newstitle,content,abstract,publish_date,meta_keywords,label
0,623,post,https://www.factcheck.org/2020/07/face-mask-ex...,https://www.facebook.com/tednugent/photos/a.10...,A “face mask exempt card” shared on social med...,"""Ted Nugent""",snag limbrats that daddy 22s out of the toweri...,ted nugent added a new photo ..,NaN,"""""",1
1,624,post,https://www.factcheck.org/2020/07/video-presen...,https://www.facebook.com/jeff.neff.528/videos/...,A test on video with a gas detector shows that...,"""Jeff""",see more of jeff neff on facebook.,face masks you decide. this is worth a few min...,NaN,"""""",1
2,625,post,https://www.factcheck.org/2020/06/painting-of-...,https://twitter.com/SexCounseling/status/12764...,A painting depicting children in face masks wa...,"""""",we 've detected that javascript is disabled in...,NaN,NaN,"""""",1
3,626,article,https://healthfeedback.org/claimreview/contrar...,https://www.wakingtimes.com/2020/05/15/bill-ga...,According to Bill Gates the COVID-19 RNA vacci...,"""403 Forbidden""",nan,NaN,NaN,"""""",1
4,627,post,https://www.factcheck.org/2020/06/meme-misrepr...,https://www.facebook.com/photo.php?fbid=424048...,Meme shows Florida Surgeon General saying: Eve...,"""Jeff Cloud""",see more of jeff cloud on facebook.,it 's all coming out now .....,NaN,"""""",1
5,628,post,https://www.politifact.com/factchecks/2020/jun...,https://www.facebook.com/photo.php?fbid=101569...,Alexandria Ocasio-Cortez tweeted “It’s vital t...,"""Theresa Ivancik""",see more of theresa ivancik on facebook.,NaN,NaN,"""""",1
6,629,post,https://www.politifact.com/factchecks/2020/jun...,https://twitter.com/realDonaldTrump/status/127...,COVID-19 cases are “up only because of our big...,"""""",we 've detected that javascript is disabled in...,NaN,NaN,"""""",1
7,630,post,https://www.politifact.com/factchecks/2020/jun...,https://www.facebook.com/photo.php?fbid=153586...,If you are licensed to carry a concealed weapo...,"""Chad Hernandez""",see more of chad hernandez on facebook.,[],NaN,"""""",1
8,631,post,https://www.factcheck.org/2020/06/fake-aoc-twe...,https://twitter.com/PhilMcCrackin44/status/127...,Claim that Rep. Alexandria Ocasio-Cortez tweet...,"""""",we 've detected that javascript is disabled in...,NaN,NaN,"""""",1
9,633,post,https://www.politifact.com/factchecks/2020/jun...,https://www.instagram.com/p/CBxutmXAT76/?utm_s...,A “face mask exempt card” allows you to cite t...,"""@alphagrown17 on : “🗣I DIDN’T MAKE THIS I’M J...",nan,"4,050 likes 484 comments @alphagrown17 on inst...",NaN,"""""",1


In [29]:
df.columns

Index(['Unnamed: 0', 'type', 'fact_check_url', 'news_url', 'title',
       'newstitle', 'content', 'abstract', 'publish_date', 'meta_keywords',
       'label'],
      dtype='object')

In [30]:
df = df.rename(columns = {'Unnamed: 0':'ID'})

In [31]:
df.head(2)

,ID,type,fact_check_url,news_url,title,newstitle,content,abstract,publish_date,meta_keywords,label
0,623,post,https://www.factcheck.org/2020/07/face-mask-ex...,https://www.facebook.com/tednugent/photos/a.10...,A “face mask exempt card” shared on social med...,"""Ted Nugent""",snag limbrats that daddy 22s out of the toweri...,ted nugent added a new photo ..,NaN,"""""",1
1,624,post,https://www.factcheck.org/2020/07/video-presen...,https://www.facebook.com/jeff.neff.528/videos/...,A test on video with a gas detector shows that...,"""Jeff""",see more of jeff neff on facebook.,face masks you decide. this is worth a few min...,NaN,"""""",1


**Dropping more useless columns**

In [32]:
#dropping abstract as well because the count of na values in abstract is just too many 610/1393 (44%)
count = df["abstract"].isna().sum()
count

594

In [33]:
df = df.drop(['fact_check_url', 'news_url','abstract'] , axis='columns')

In [34]:
# making pandas display as many rows as I want
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df.head(100)

,ID,type,title,newstitle,content,publish_date,meta_keywords,label
0,623,post,A “face mask exempt card” shared on social med...,"""Ted Nugent""",snag limbrats that daddy 22s out of the toweri...,NaN,"""""",1
1,624,post,A test on video with a gas detector shows that...,"""Jeff""",see more of jeff neff on facebook.,NaN,"""""",1
2,625,post,A painting depicting children in face masks wa...,"""""",we 've detected that javascript is disabled in...,NaN,"""""",1
3,626,article,According to Bill Gates the COVID-19 RNA vacci...,"""403 Forbidden""",nan,NaN,"""""",1
4,627,post,Meme shows Florida Surgeon General saying: Eve...,"""Jeff Cloud""",see more of jeff cloud on facebook.,NaN,"""""",1
5,628,post,Alexandria Ocasio-Cortez tweeted “It’s vital t...,"""Theresa Ivancik""",see more of theresa ivancik on facebook.,NaN,"""""",1
6,629,post,COVID-19 cases are “up only because of our big...,"""""",we 've detected that javascript is disabled in...,NaN,"""""",1
7,630,post,If you are licensed to carry a concealed weapo...,"""Chad Hernandez""",see more of chad hernandez on facebook.,NaN,"""""",1
8,631,post,Claim that Rep. Alexandria Ocasio-Cortez tweet...,"""""",we 've detected that javascript is disabled in...,NaN,"""""",1
9,633,post,A “face mask exempt card” allows you to cite t...,"""@alphagrown17 on : “🗣I DIDN’T MAKE THIS I’M J...",nan,NaN,"""""",1


**from above we see that there issnt any funny characters or html tags or artifacts that we need to get rid in the next step**

In [35]:
df.shape

(1377, 8)

In [36]:
# no duplicate rows
df = df.drop_duplicates()

In [37]:
df.shape

(1377, 8)

**We start to clean the 'title' and 'content' columns**

In [38]:
def preprocess(ReviewText):
    ReviewText = ReviewText.str.replace("(<br/>)", "")
    ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
    ReviewText = ReviewText.str.replace('(&amp)', '')
    ReviewText = ReviewText.str.replace('(&gt)', '')
    ReviewText = ReviewText.str.replace('(&lt)', '')
    ReviewText = ReviewText.str.replace('(\xa0)', ' ')  
    return ReviewText
df['Preproc_title'] = preprocess(df['title'])
df['Preproc_content'] = preprocess(df['content'])


#df['content_len'] = df['text'].astype(str).apply(len) #I suppose we dont need this but im just include just in case
#df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

In [39]:
def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct
df['Preproc_title'] = preprocess(df['title']).apply(lambda x: remove_punctuation(x))
df['Preproc_content']= preprocess(df['content']).apply(lambda x: remove_punctuation(x))

df.head()
#check why line 0 has ""

,ID,type,title,newstitle,content,publish_date,meta_keywords,label,Preproc_title,Preproc_content
0,623,post,A “face mask exempt card” shared on social med...,"""Ted Nugent""",snag limbrats that daddy 22s out of the toweri...,NaN,"""""",1,A “face mask exempt card” shared on social med...,snag limbrats that daddy 22s out of the toweri...
1,624,post,A test on video with a gas detector shows that...,"""Jeff""",see more of jeff neff on facebook.,NaN,"""""",1,A test on video with a gas detector shows that...,see more of jeff neff on facebook
2,625,post,A painting depicting children in face masks wa...,"""""",we 've detected that javascript is disabled in...,NaN,"""""",1,A painting depicting children in face masks wa...,we ve detected that javascript is disabled in ...
3,626,article,According to Bill Gates the COVID-19 RNA vacci...,"""403 Forbidden""",nan,NaN,"""""",1,According to Bill Gates the COVID19 RNA vaccin...,nan
4,627,post,Meme shows Florida Surgeon General saying: Eve...,"""Jeff Cloud""",see more of jeff cloud on facebook.,NaN,"""""",1,Meme shows Florida Surgeon General saying Ever...,see more of jeff cloud on facebook


In [40]:
# def tokenize(text):
#     split=re.split("\W+",text) 
#     return split
# df['Preproc_title']=df['Preproc_title'].apply(lambda x: tokenize(x.lower()))
# df['Preproc_content']=df['Preproc_content'].apply(lambda x: tokenize(x.lower()))
# df.head()

In [41]:
# stopword = nltk.corpus.stopwords.words('english')
# print(stopword[:11])

In [42]:
# def remove_stopwords(text):
#     text=[word for word in text if word not in stopword]
#     return text
# df['Preproc_title'] = df['Preproc_title'].apply(lambda x: remove_stopwords(x))
# df['Preproc_content'] = df['Preproc_content'].apply(lambda x: remove_stopwords(x))
# df.head()

In [43]:
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [44]:
df['label'].iplot(
    kind='hist',
    xTitle='review',
    linecolor='black',
    yTitle='count',
    title='Review Rating Distribution')

In [45]:
#x = vec.fit_transform(corpus['Preproc_content'].values.astype('U'))  ## Even astype(str) would work
df.dtypes

ID                  int64
type               object
title              object
newstitle          object
content            object
publish_date       object
meta_keywords      object
label               int64
Preproc_title      object
Preproc_content    object
dtype: object

In [46]:
df.isnull().sum()

ID                    0
type                  0
title                 0
newstitle             2
content               0
publish_date       1027
meta_keywords         2
label                 0
Preproc_title         0
Preproc_content       0
dtype: int64

In [47]:
#df.fillna('0')
##df['Preproc_content'] = df['Preproc_content'].values.astype('U')

In [48]:
df.isnull().sum()

ID                    0
type                  0
title                 0
newstitle             2
content               0
publish_date       1027
meta_keywords         2
label                 0
Preproc_title         0
Preproc_content       0
dtype: int64

In [50]:
df.publish_date.value_counts()

2/11/20                      202
4/30/20                       71
6/11/20                        2
5/4/20                         2
7/24/20                        2
2020-04-30T21:31:54+00:00      2
2020-04-25T11:32:32+00:00      2
6/12/20                        1
5/11/20                        1
2020-05-07T20:30:53-04:00      1
2020-07-11T03:06:01+00:00      1
4/25/20                        1
4/24/20                        1
28-Apr-20                      1
6/16/20                        1
2020-06-14T04:58:31+00:00      1
6/14/20                        1
2020-05-22T16:49:14-04:00      1
2020-06-20T09:59:33-04:00      1
6/18/20                        1
27-Apr-20                      1
5/12/20                        1
2020-04-29T19:39:22.000Z       1
2020-05-06T11:37:38Z           1
2020-05-05T10:21:04-04:00      1
2020-04-28T17:17:14+00:00      1
2020-06-15T14:04:38-04:00      1
2020-05-18T00:17:00-04:00      1
2020-06-11T08:46:29-04:00      1
8-Jun-20                       1
2020-05-12

In [51]:
df.meta_keywords.value_counts()

""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    846
"video, sharing, camera phone, video phone, free, upload"                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [49]:
# unigram words after removing stopwords

common_words4 = get_top_n_words(df['Preproc_content'], 20)
for word, freq in common_words4:
    print(word, freq)
df4 = pd.DataFrame(common_words4, columns = ['Preproc_content' , 'count'])
df4.groupby('Preproc_content').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in review after removing stop words')

covid19 876
protect 401
using 388
service 371
online 367
people 365
website 358
attacks 355
security 355
data 284
health 276
coronavirus 265
information 240
new 199
disease 183
virus 181
pandemic 177
time 168
nan 166
available 166
